# Homework 10

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from collections import namedtuple
from itertools import chain

# Take a moment to review the namedtuple documentation:
# https://docs.python.org/3/library/collections.html#collections.namedtuple

## Common logic data structures
Not = namedtuple("Not", ["sentence"])
And = namedtuple("And", ["sentence1", "sentence2"])
Or = namedtuple("Or", ["sentence1", "sentence2"])
Implies = namedtuple("Implies", ["sentence1", "sentence2"])

## Propositional logic data structures
Proposition = str # Name of the proposition
PropositionalModel = dict # Proposition -> bool

## Example of PropositionalModel, used in tests
IS_RAINING = Proposition("is-raining")
IS_SUNNY = Proposition("is-sunny")
NEED_UMBRELLA = Proposition("need-umbrella")
PROP_MODEL = PropositionalModel({
  IS_RAINING: True,
  IS_SUNNY: False,
  NEED_UMBRELLA: True,
})

## First-order logic data structures
Object = str
Constant = namedtuple("Constant", ["name"])
Variable = namedtuple("Variable", ["name"])
Predicate = namedtuple("Predicate", ["name", "arity"])
Atom = namedtuple("Atom", ["predicate", "terms"])
ForAll = namedtuple("ForAll", ["variable", "sentence"])
Exists = namedtuple("Exists", ["variable", "sentence"])
Interpretation = namedtuple("Interpretation", ["constant_map", "predicate_map"])
FOLModel = namedtuple("FOLModel", ["objects", "interpretation"])

# Example of FOLModel, used in tests
TOM = Object("tom-object")
NOMSY = Object("nomsy-object")
PUDDLES = Object("puddles-object")
OBJECTS = {TOM, NOMSY, PUDDLES}
CT, CN, CP = Constant("Tom"), Constant("Nomsy"), Constant("Puddles")
X, Y = Variable("X"), Variable("Y")
Likes = Predicate("Likes", 2)
IsDog = Predicate("IsDog", 1)
CONSTANT_MAP = {CT : TOM, CN: NOMSY, CP: PUDDLES}
PREDICATE_MAP = {
  Likes: {(TOM, NOMSY), (TOM, PUDDLES), (NOMSY, TOM)},
  IsDog: {(NOMSY,), (PUDDLES,)},
}
INTERPRETATION = Interpretation(constant_map=CONSTANT_MAP,
                                predicate_map=PREDICATE_MAP)
FOL_MODEL = FOLModel(objects=OBJECTS, interpretation=INTERPRETATION)

## First-order CNF logical data structures
Literal = namedtuple("Literal", ["atom", "is_positive"])
def negate(literal):
  return Literal(literal.atom, not literal.is_positive)
def Clause(literals=tuple()):
  return frozenset(literals)
def CNFSentence(clauses):
  return set(clauses)

def unify_var(v, t, th):
  """Unify a variable and a (non-function) term.

  Args:
    v: A Variable.
    t: A Constant or Variable.
    th: A dict mapping variables to objects.

  Returns:
    theta: A dict mapping variables to objects
      or None if no substitution exists.
  """
  if th is None: return None
  if v == t:
    return th
  elif v in th:
    return unify(th[v], t, th)
  elif t in th:
    return unify_var(v, th[t], th)
  else:
    return compose_subst(th, {v:t})

def compose_subst(s1, s2):
  return dict(chain(((x, subst(t, s2)) for (x, t) in s1.items() if x != subst(t, s2)),
                    ((x, t) for (x, t) in s2.items() if not x in s1)))


def subst(a, th):
  """Substitute variables for a single Literal.

  Args:
    a: A Constant, Variable, Atom, Literal, list, tuple, or frozenset.
    th: A dict mapping variables to objects.

  Returns:
    b: Same type as a.
  """
  if isinstance(a, Constant):
    return a
  elif isinstance(a, Variable):
    return th[a] if a in th else a
  elif isinstance(a, Atom):
    return Atom(a.predicate, subst(a.terms, th))
  elif isinstance(a, Literal):
    return Literal(subst(a.atom, th), a.is_positive)
  elif isinstance(a, (list, tuple)):
    return tuple(subst(x, th) for x in a)
  elif isinstance(a, frozenset):
    return frozenset(subst(x, th) for x in a)
  else:
    raise Exception('Unknown type:'+str(a))


def unify(a, b, th):
  """Run unification.

  Args:
    a: A Constant, Variable, Atom, Literal, or list or tuple of terms.
    b: A Constant, Variable, Atom, Literal, or list or tuple of terms.
    th: A dict mapping variables to objects, or None if no unifier exists.

  Returns:
    new_th: A new dict representing a unifier, or None if none exist.
  """
  if th is None: return None
  if isinstance(a, Constant):
    if isinstance(b, Variable):
      return unify_var(b, a, th)
    else:
      return th if a == b else None
  elif isinstance(a, Variable):
    return unify_var(a, b, th)
  elif isinstance(b, Variable):
    return unify_var(b, a, th)
  elif isinstance(a, Atom):
    return unify(a.terms, b.terms, th) if (isinstance(b, Atom) and a.predicate == b.predicate) else None
  elif isinstance(a, Literal):
    return unify(a.atom, b.atom, th) if (isinstance(b, Literal) and a.is_positive == b.is_positive) else None
  elif isinstance(a, (list, tuple)) and isinstance(b, (list, tuple)):
    if len(a) == 0 or len(b) == 0:
      return th if len(a) == len(b) else None
    else:
      return unify(a[0], b[0], unify(a[1:], b[1:], th))
  else:
    raise Exception('Unknown type:'+str(a))



## Atom Evaluation


### Question
*Note: for these questions, refer to the top of the Colab notebook.*
Write a function that takes a FOL atom and evaluates it against a single model.

For reference, our solution is **10** line(s) of code.

In [ ]:
def evaluate_atom(atom, model, substitution):
    """Evaluate if an atom holds under the model.

    Args:
      atom: An Atom.
      model: A FOLModel.
      substitution: A dict mapping variables to objects.

    Returns:
      holds: A bool.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def evaluate_atom_test1():
  assert evaluate_atom(Atom(IsDog, (CN,)), FOL_MODEL, {}) == True

evaluate_atom_test1()


def evaluate_atom_test2():
  assert evaluate_atom(Atom(IsDog, (X,)), FOL_MODEL, {X: NOMSY}) == True

evaluate_atom_test2()


def evaluate_atom_test3():
  assert evaluate_atom(Atom(IsDog, (CT,)), FOL_MODEL, {}) == False

evaluate_atom_test3()


def evaluate_atom_test4():
  assert evaluate_atom(Atom(IsDog, (X,)), FOL_MODEL, {X: TOM}) == False

evaluate_atom_test4()


def evaluate_atom_test5():
  assert evaluate_atom(Atom(Likes, (X, Y)), FOL_MODEL, {X: TOM, Y: NOMSY}) == True

evaluate_atom_test5()

print('Tests passed.')

## First-order Logic Sentence Evaluation


### Question
Use your implementation of evaluate_atom to complete the following implementation of FOL sentence evaluation.

For reference, our solution is **37** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `evaluate_atom`. You may not need to use all of them.

In [ ]:
def evaluate_fol_sentence(sentence, model, substitution=None):
    """Evaluate a first-order logic sentence against a single model.

    Note that Literals are not used here (we use them in later problems).

    Be careful about updating `substitution` recursively. You may want
    to create a copy of the dict (`substitution.copy()`) before each
    recursive call.

    Args:
      sentence: An Atom, And, Or, Not, Implies, ForAll, or Exists.
      model: A FOLModel.
      substitution: A dict mapping variables to objects, or None,
        representing an empty dict.

    Returns:
      holds: A bool representing the truth value of the sentence
        under the model.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def evaluate_fol_sentence_test1():
  assert evaluate_fol_sentence(And(Atom(IsDog, (CN,)), Atom(IsDog, (CP,))), FOL_MODEL) == True

evaluate_fol_sentence_test1()


def evaluate_fol_sentence_test2():
  assert evaluate_fol_sentence(Or(Atom(IsDog, (CT,)), Not(Atom(IsDog, (CP,)))), FOL_MODEL) == False

evaluate_fol_sentence_test2()


def evaluate_fol_sentence_test3():
  assert evaluate_fol_sentence(And(Atom(IsDog, (CN,)), Atom(IsDog, (CP,))), FOL_MODEL) == True

evaluate_fol_sentence_test3()


def evaluate_fol_sentence_test4():
  assert evaluate_fol_sentence(Exists(X, And(Atom(IsDog, (X,)), Not(Atom(Likes, (CT, X))))), FOL_MODEL) == False

evaluate_fol_sentence_test4()


def evaluate_fol_sentence_test5():
  assert evaluate_fol_sentence(Exists(X, Exists(Y, Atom(Likes, (Y, X)))), FOL_MODEL) == True

evaluate_fol_sentence_test5()

print('Tests passed.')

## FOL Binary Resolution


### Question
Complete the following implementation of first-order binary resolution.
Given two clauses, return all possible new clauses that result from applying the binary resolution rule.

<b>Hint 1</b>: The helper function `unify` may return an empty dict for certain inputs. Note that this means the inputs
can be successfully unified without any variable substitution.

<b>Hint 2</b>: In your code, you can use `if unify(a, b, {}) is not None` to test if the unification is successful.


For reference, our solution is **8** line(s) of code.

In [ ]:
def binary_resolution(clause1, clause2):
    """Return all new clauses resulting from binary resolution.

    Args:
      clause1: A Clause (frozenset of Literals).
      clause2: A Clause (frozenset of Literals).

    Returns:
      clauses: A set of new Clauses.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
clause1 = Clause([Literal(Atom(IsDog, (CT,)), True)])
clause2 = Clause([Literal(Atom(IsDog, (CT,)), False)])
assert binary_resolution(clause1, clause2) == {Clause()}


clause1 = Clause([Literal(Atom(IsDog, (X,)), True)])
clause2 = Clause([Literal(Atom(IsDog, (Y,)), False)])
assert binary_resolution(clause1, clause2) == {Clause()}


clause1 = Clause([Literal(Atom(IsDog, (X,)), True)])
clause2 = Clause([Literal(Atom(IsDog, (Y,)), True)])
assert binary_resolution(clause1, clause2) == set()


# All dogs are liked by Tom
clause1 = Clause([Literal(Atom(IsDog, (X,)), False), Literal(Atom(Likes, (CT, X)), True)])
# Nomsy is a dog
clause2 = Clause([Literal(Atom(IsDog, (CN,)), True)])
# So Nomsy must be liked by Tom
assert binary_resolution(clause1, clause2) == {Clause([Literal(Atom(Likes, (CT, CN)), True)])}

print('Tests passed.')

## FOL Resolution Prover


### Question
Complete the following implementation of a first-order resolution prover. Given a sentence in CNF form, and a single query clause, check if the sentence entails the query. See unit tests for examples.

For reference, our solution is **17** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `binary_resolution`. You may not need to use all of them.

In [ ]:
def resolution_prover(kb, q):
    """Check if a knowledge base entails a query.

    That is, if kb ^ not q entails False.
    That is, if we can prove False from kb ^ not q.
    May run forever.

    Args:
      kb: A CNFSentence.
      q: A single Clause.

    Returns:
      entails: True if kb entails q.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
# All dogs are liked by Tom
kb_clause1 = Clause([Literal(Atom(IsDog, (X,)), False), Literal(Atom(Likes, (CT, X)), True)])
# Nomsy is a dog
kb_clause2 = Clause([Literal(Atom(IsDog, (CN,)), True)])
kb = CNFSentence([kb_clause1, kb_clause2])
# Tom likes Nomsy
query = Clause([Literal(Atom(Likes, (CT, CN)), True)])
assert resolution_prover(kb, query) == True


# Tom does not like Nomsy (impossible!)
query = Clause([Literal(Atom(Likes, (CT, CN)), False)])
assert resolution_prover(kb, query) == False


# Russell & Norvig example
American = Predicate("American", 1)
Weapon = Predicate("Weapon", 1)
Sells = Predicate("Sells", 3)
Hostile = Predicate("Hostile", 1)
Criminal = Predicate("Criminal", 1)
Missile = Predicate("Missile", 1)
Enemy = Predicate("Enemy", 1)
Owns = Predicate("Owns", 2)
Nono = Constant("Nono")
America = Constant("America")
West = Constant("West")
M1 = Constant("M1")

clause1 = Clause([
  Literal(Atom(American, (Variable("x1"),)), False),
  Literal(Atom(Weapon, (Variable("y1"),)), False),
  Literal(Atom(Sells, (Variable("x1"),Variable("y1"),Variable("z1"))), False),
  Literal(Atom(Hostile, (Variable("z1"),)), False),
  Literal(Atom(Criminal, (Variable("x1"),)), True),
])
clause2 = Clause([
  Literal(Atom(Missile, (Variable("x2"),)), False),
  Literal(Atom(Owns, (Nono, Variable("x2"),)), False),
  Literal(Atom(Sells, (West, Variable("x2"), Nono)), True),
])
clause3 = Clause([
  Literal(Atom(Enemy, (Variable("x3"), America)), False),
  Literal(Atom(Hostile, (Variable("x3"),)), True),
])
clause4 = Clause([
  Literal(Atom(Missile, (Variable("x4"),)), False),
  Literal(Atom(Weapon, (Variable("x4"),)), True),
])
clause5 = Clause([
  Literal(Atom(Owns, (Nono, M1)), True),
])
clause6 = Clause([
  Literal(Atom(Missile, (M1,)), True),
])
clause7 = Clause([
  Literal(Atom(American, (West,)), True),
])
clause8 = Clause([
  Literal(Atom(Enemy, (Nono, America)), True),
])
kb = CNFSentence([clause1, clause2, clause3, clause4, clause5, clause6, clause7, clause8])
query = Clause([
  Literal(Atom(Criminal, (West,)), True),
])
assert resolution_prover(kb, query) == True

print('Tests passed.')